# Quick Webscraping Script Matching Cities to Counties
## Alex Bass

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from functools import reduce
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd

In [2]:
data = pd.read_csv('shoot.csv')
data.sort_values('State', inplace=True)
data = data[['City', 'State']].reset_index()

In [3]:
link = 'https://www.stats.indiana.edu/uspr/b/place_frame.html'
s = Service(ChromeDriverManager().install())
options = Options()
options.headless = True
driver = webdriver.Chrome(service=s, options=options)
driver.get(link)
driver.switch_to.frame("place_bar")

counties = []
row_order = []

for i in range(51):
    a = driver.find_element(By.NAME, 'states')
    drop = Select(a)
    txt_match = drop.options[i].text
    drop.select_by_visible_text(txt_match)
    for j,k in enumerate(data['State']):
        if txt_match == k:
            inpt_txt = data['City'][j]
            driver.find_element(By.NAME, 'place_name').send_keys(inpt_txt)
            time.sleep(0.5)
            driver.find_element(By.NAME, 'Submit').click()
            time.sleep(0.5)
            driver.find_element(By.NAME, 'place_name').clear()
            driver.switch_to.default_content()
            driver.switch_to.frame("place_results")
            try:
                txt_append = driver.find_elements(By.TAG_NAME, 'td')[2].text
            except:
                txt_append = 'NA'
            counties.append(txt_append)
            row_order.append(data['index'][j])
            driver.switch_to.default_content()
            driver.switch_to.frame("place_bar")

driver.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/alex/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [4]:
data = data.query('State != "VI"').reset_index()
data = data[['City','State']]

In [5]:
matched_data = pd.DataFrame({
    'counties' : counties,
    'index' : row_order
})

matched_data.to_csv('matched_data.csv')

In [13]:
matched_data = matched_data.sort_values('index').reset_index()
matched_data

,level_0,counties,index
0,79,"Pima County, AZ",0
1,1648,"Orangeburg County, SC",1
2,991,"Monroe County, MI",2
3,749,"LaPorte County, IN",3
4,726,NA,4
...,...,...,...
2061,772,"Lake County, IN",2062
2062,983,"Monroe County, MI",2063
2063,105,"Pulaski County, AR",2064
2064,6,"Etowah County, AL",2065


In [8]:
data1 = pd.read_csv('shoot.csv').query('State != "VI"').reset_index()

In [17]:
data2 = pd.concat([data1, matched_data], axis=1)

In [21]:
county_match = pd.read_csv('county_pop.csv')
county_match

,countyFIPS,CountyName,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394
...,...,...,...,...
3190,56037,Sweetwater County,WY,42343
3191,56039,Teton County,WY,23464
3192,56041,Uinta County,WY,20226
3193,56043,Washakie County,WY,7805


In [23]:
county_match = county_match.query('CountyName != "Statewide Unallocated"').reset_index(drop=True)

In [25]:
data2.counties.str.split

,index,Incident_ID,Sources,Number_News,Media_Attention,Reliability,Date,Quarter,School,City,...,Officer_Involved,Bullied,Domestic_Violence,Gang_Related,Preplanned,Shots_Fired,Active_Shooter_FBI,level_0,counties,index
0,0,20200727AZCAO,https://www.kgun9.com/news/education/man-dies-...,3,Local,4,2020-07-27,Summer,Canyon del Oro High School,Oro Valley,...,No,No,No,No,No,NaN,No,79,"Pima County, AZ",0
1,1,20200721SCBRO,https://abcnews4.com/news/crime-news/person-of...,2,Local,3,2020-07-21,Summer,Brookdale Elementary School,Orangeburg,...,No,No,No,NaN,No,NaN,No,1648,"Orangeburg County, SC",1
2,2,20200714MIWID,https://www.detroitnews.com/story/news/local/m...,1,Local,3,2020-07-14,Summer,Willow Woods Elementary School,Detroit,...,No,No,No,No,No,NaN,No,991,"Monroe County, MI",2
3,3,20200704INLAM,https://www.abc57.com/news/murder-suspect-char...,2,Local,4,2020-07-04,Summer,Lake Hills Elementary School,Michigan City,...,No,No,No,No,No,NaN,No,749,"LaPorte County, IN",3
4,4,20200701ILFRP,https://www.dailyherald.com/news/20200714/pros...,1,Local,3,2020-07-01,Summer,Frost Elementary School,Prospect Heights,...,No,No,No,Yes,No,NaN,No,726,NA,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2061,2062,20220605INWEG,https://abc7chicago.com/gary-indiana-shooting-...,2,Local,4,2022-06-05,Summer,West Side Leadership Academy,Gary,...,No,No,No,NaN,No,99,No,772,"Lake County, IN",2062
2062,2063,20220607MIPED,https://www.wxyz.com/news/shots-fired-at-persh...,4,Local,4,2022-06-07,Summer,Pershing High School,Detroit,...,No,No,No,NaN,No,99,No,983,"Monroe County, MI",2063
2063,2064,20220608ARLIL,https://www.kark.com/crime/lrpd-school-bus-hit...,5,National,4,2022-06-08,Summer,Little Rock School District Bus,Little Rock,...,No,No,No,NaN,No,99,No,105,"Pulaski County, AR",2064
2064,2065,20220609ALWAG,https://www.live5news.com/2022/06/09/potential...,100,National,4,2022-06-09,Summer,Walnut Park Elementary School,Gadsden,...,NaN,NaN,NaN,NaN,NaN,NaN,No,6,"Etowah County, AL",2065


In [28]:
data2[['counties','state2']] = data2.counties.str.split(', ', expand = True)

In [33]:
data3 = pd.merge(data2, county_match, left_on=['State', 'counties'], right_on=['State', 'CountyName'], how='left')

In [36]:
data3 = data3.drop(['index', 'level_0', 'counties', 'state2'], axis = 1)

In [37]:
data3.to_csv('data_w_county_match.csv')